In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive



/content/drive/MyDrive


In [ ]:
# 2. 라이브러리 설치
!pip install opencv-python mediapipe numpy pandas scikit-learn tensorflow


ERROR: Operation cancelled by user


In [4]:
# 경로 설정
DATASET_PATH = "/content/drive/MyDrive/Game_CapstoneDesign/Game-CapstoneDesign"
VIDEO_PATH = f"{DATASET_PATH}/data/top100_videos_only"
TEST_VIDEO_PATH = f"{DATASET_PATH}/data/test_videos"
LABEL_EXCEL = f"{DATASET_PATH}/data/KETI_top100_clean_words.xlsx"
TEST_LABEL_EXCEL = f"{DATASET_PATH}/data/test_data.xlsx"


In [5]:
# 01. label 불러오기 및 영상번호 매핑
import pandas as pd
import os
import unicodedata

def load_label_info(label_excel_path):
    label_df = pd.read_excel(label_excel_path)
    label_df['파일ID'] = label_df['파일명'].apply(lambda x: os.path.splitext(x)[0])
    video_to_label = dict(zip(label_df['파일ID'], label_df['한국어']))
    labels = sorted(unicodedata.normalize("NFC", l) for l in label_df['한국어'].unique())
    label_map = {label: idx for idx, label in enumerate(labels)}
    return label_df, label_map, video_to_label

label_df, label_map, video_to_label = load_label_info(LABEL_EXCEL)
test_label_df, test_label_map, test_video_to_label = load_label_info(TEST_LABEL_EXCEL)

In [ ]:
!pip uninstall mediapipe -y
!pip install mediapipe==0.10.5


Found existing installation: mediapipe 0.10.5
Uninstalling mediapipe-0.10.5:
  Successfully uninstalled mediapipe-0.10.5
  Using cached mediapipe-0.10.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
Using cached mediapipe-0.10.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (33.5 MB)


In [ ]:
# 02. MediaPipe 키포인트 추출 함수
def extract_keypoints(results):
    import numpy as np
    pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [ ]:
# 03. 영상 → 프레임 단위 npy 저장
import cv2
import mediapipe as mp
import numpy as np
import os
from tqdm import tqdm

#TRAIN_SAVE_PATH = f"{DATASET_PATH}/data/MP_Data" #키포인트 .npy를 저장할 폴더
#os.makedirs(TRAIN_SAVE_PATH, exist_ok=True)
TEST_SAVE_PATH = f"{DATASET_PATH}/data/MP_Data_Test"
os.makedirs(TEST_SAVE_PATH, exist_ok=True)
mp_holistic = mp.solutions.holistic

def process_videos(video_dir, save_dir_root, video_to_label):
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        for file in tqdm(os.listdir(video_dir), desc=f"Processing {video_dir}"):
            if not file.lower().endswith(('.mp4', '.avi', '.mov')):
                continue

            filename = os.path.splitext(file)[0]
            label = video_to_label.get(filename)
            if label is None:
                continue

            cap = cv2.VideoCapture(os.path.join(video_dir, file))
            save_dir = os.path.join(save_dir_root, label, filename)
            os.makedirs(save_dir, exist_ok=True)

            frame_count = 0
            while cap.isOpened():
              ret, frame = cap.read()
              if not ret or frame is None:
                  break

              try:
                  image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                  image = np.ascontiguousarray(image, dtype=np.uint8)
                  results = holistic.process(image)
                  keypoints = extract_keypoints(results)
                  np.save(os.path.join(save_dir, f"{frame_count}.npy"), keypoints)
                  frame_count += 1

              except Exception as e:

                  break
            cap.release()

# 실행

#process_videos(VIDEO_PATH, TRAIN_SAVE_PATH, video_to_label)
process_videos(TEST_VIDEO_PATH, TEST_SAVE_PATH, test_video_to_label)

ModuleNotFoundError: No module named 'mediapipe'

#프레임 60개, 샘플링할때



#Prototypical Networks 이용


#원래 모델 구조
